In [1]:
from collections import Counter, defaultdict
from itertools import zip_longest
from chyp.polynomial import * 
from lark import Lark, Transformer


# Define the Lark grammar for parsing any polynomial expression with general variable names
grammar = r"""
    ?start: expr              
    
    ?expr: term (("+" term) | sub_term)*   -> add
    ?sub_term: ("-" term) -> sub
    
    term: num                              
         | monomial             
         | num "*" monomial

    
    monomial: var_term ("*" var_term)*
    ?var_term: var 
             | var "^" num -> pow
             

    var: IDENT -> var
    
    num : INT -> number
        | "-"INT -> neg_number
        
    IDENT: ("_"|LETTER) ("_"|"."|LETTER|DIGIT)*
    
    %import common.LETTER
    %import common.DIGIT
    %import common.HEXDIGIT
    %import common.INT
    %import common.WS
    %import common.SH_COMMENT
    %ignore WS
    %ignore SH_COMMENT
"""


# Transformer class to convert parse tree into Polynomial terms
class PolynomialTransformer(Transformer):
    def __init__(self):
        super().__init__()
        self.vars = []
        self.nvars = 0

    def number(self, n):
        value = int(n[0])
        return int(n[0])

    def neg_number(self, n):
        value = int(n[0])
        return -int(n[0])

    def var(self, v):
        var = v[0][0]

        if var not in self.vars: 
            self.vars.append(var)
            self.nvars += 1
        
        return (var, 1)

    def pow(self, args):
        var = args[0][0]
        power = args[0][1]
        new_power = args[1]
        
        return (var, power + new_power - 1)

    def add(self, args):
        return Polynomial(args, subs=self.vars)

    def sub(self, args):
        monomial = args[0][0]
        coeff = args[0][1] 
        
        return (monomial, -coeff)
        

    def term(self, args):
        if len(args) == 1: 
            a = args[0]
            if isinstance(a, int): 
                # The term only has a coefficient 
                return ((0,), a)
            else: 
                # The term only has a monomial 
                return (a, 1)

        # The term has a coefficient and a monomial 
        coeff = args[0]
        monomial = args[1]
            
        return (monomial, coeff)

    def monomial(self, args):
        c = Counter() 
        for var, power in args: 
            c[var] += power
    
        # Get the variable indices and their respective powers 
        # we need to do everything in reverse order because 
        # we use zip_longest to pair variables to their powers 
        # and we always want to start with the latest variable
        vs     = reversed(range(self.nvars))
        powers = map(lambda x: c[x], reversed(c.keys()))

        # Pair the variables to their powers, filling in 0 if the 
        # variable does not occur in this monomial. 
        # Then take only the power
        exps = map(lambda x: x[1], # Take the second value
                   zip_longest(vs, powers, fillvalue=0))

        # We need to put back everything into the correct order
        exps = list(reversed(list(exps)))
        
        return tuple(exps)



# Define the Lark parser
parser = Lark(grammar, parser="lalr")

# Function to parse an expression and return a Polynomial object
def parse(expression):
    tree = parser.parse(expression)
    transformer = PolynomialTransformer()
    return transformer.transform(tree)

In [2]:
p = parse("1 + 3 * x * y^2 + z + z^2 + z^2")
p1 = parse("1 + z + z^2 + z^2 + 3 * x * y^2")

print(p.terms)
print(p1.terms)

show(p, p1)

# Because this substitutes variables
# as  x = 1, y = 1, z = 8 in p 
# but z = 1, x = 1, y = 8 in p1
p(1, 1, 8) == p1(1, 1, 8)

p == p1

Counter({(1, 2): 3, (0, 0, 2): 2, (0,): 1, (0, 0, 1): 1})
Counter({(0, 1, 2): 3, (2,): 2, (0,): 1, (1,): 1})


<IPython.core.display.Math object>

<IPython.core.display.Math object>

True

In [3]:
p = parse("1") 

show(p)

<IPython.core.display.Math object>